In [ ]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
random_seed = 2020

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xgboost as xgb

#Case 1: Text Sentiment Classification


*   inputs:  imdb 영화 리뷰
*   outputs: 긍정(1), 부정(0)



#Load dataset

In [ ]:
filepath = '/content/drive/MyDrive/데이터마이닝/LAB5/week12_ml/week12_ml/dataset/imdb_dataset.txt'

In [ ]:
movie_review = pd.read_csv(filepath, delimiter='\t', compression='zip').reset_index(drop=True).dropna()
movie_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [ ]:
movie_review.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
movie_review['label'] = [0 if sentiment=='negative' else 1 for sentiment in movie_review['sentiment']]

In [ ]:
movie_review.head()

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1




*   긍정 리뷰와 부정 리뷰 비율 확인(다를 경우, 클래스 불균형 문제 발생)



In [ ]:
pd.DataFrame(movie_review['label'].value_counts(normalize=True))

,label
1,0.5
0,0.5


#Data preprocessing



*   영어 전처리



In [ ]:
! pip install contractions
import re
import contractions

In [ ]:
def text_clean(text):
    
    text = contractions.fix(text) #he's -> he is
    text = re.sub('<br />'," ",text)  
    text = re.sub('https?:/\/\S+', ' ', text) # remove urls
    text = re.sub('[0-9]+', ' ', text) # remove numbers
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\…》]+', ' ', text) # remove all symbols and punctuation except for . , ! and ?
    text = re.sub("'"," ", text)
    text = re.sub('\s+', ' ', text) # 중복 띄어쓰기 삭제

    text = text.lower() # 소문자
    return text.strip() #앞뒤 문장 공백 제거



*   레마타이징: 문장을 사전에 등록된 단어 형태로 쪼개 줌









In [ ]:
#spacy
import spacy
import en_core_web_sm #버전
from spacy.lang.en import English
from tqdm import tqdm

# https://spacy.io/usage

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) 
spacy_stopwords=spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
def spacy_lemma(text):
    text = text_clean(sentence)
    text=nlp(text)
    temp=[token.lemma_ for token in text if token.lemma_ not in ['-PRON-'] + list(spacy_stopwords)] #lemmatize한 단어가 대명사가 아니거나 불용어가 아닐 경우 추가
    
    #return " ".join(temp)
    return temp

#Split dataset

In [ ]:
#DataFrame -> list
data = movie_review['review'].values.tolist()
target = movie_review['label'].values.tolist()

In [ ]:
print(data[0], target[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [ ]:
data = [text_clean(text) for text in data]

In [ ]:
print(data[0], target[0])

one of the other reviewers has mentioned that after watching just oz episode you will be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda them city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows would not dare forget pret

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.2, 
                                                    random_state=random_seed)

In [ ]:
len(y_train)

40000

In [ ]:
len(y_test)

10000

# Word Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Counter Vectorization으로 피처 벡터화 수행
#CountVectorizer의 파라미터 설명
#lowercase: 대문자를 소문자로 변환(한글 데이터는 필요없음)
#preprocessor: 전처리 함수
#tokenizer: 문장을 형태소 단위로 변환
#min_df: 최소 5번 이상 등장하는 단어만 고려
#max_features: 최대 1600개 단어만 고려
#cnt_vect = CountVectorizer(lowercase=True, preprocessor = text_clean, tokenizer = spacy_lemma, min_df=5, max_features=100)
cnt_vect = CountVectorizer(lowercase=True, min_df=5, max_features=1600)
cnt_vect.fit(X_train)

X_train_cnt_vect = cnt_vect.transform(X_train)

#  학습 데이터로 fit()된 Vectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

In [ ]:
print(X_train_cnt_vect.shape)

(40000, 1600)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X_train
#tfidf_vect = TfidfVectorizer(lowercase=True, preprocessor = my_preprocessor,  tokenizer = spacy_lemma, min_df=5, max_features=1600)
tfidf_vect = TfidfVectorizer(lowercase=True, min_df=5, max_features=1600)

tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# X_test
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [ ]:
print(X_train_tfidf_vect.shape)

(40000, 1600)


### [+] 특성값이 높은 단어 확인

In [ ]:
max_value = X_train_cnt_vect.max(axis=0).toarray().ravel()
sorted_value = max_value.argsort()
feature_names = np.array(cnt_vect.get_feature_names())
feature_names[sorted_value[-100:]]

array(['following', 'machine', 'him', 'la', 'at', 'character', 'radio',
       'rock', 'your', 'would', 'woman', 'ray', 'buddy', 'by', 'bed',
       'story', 'never', 'lee', 'war', 'gay', 'dream', 'has', 'best',
       'one', 'robert', 'its', 'life', 'stewart', 'who', 'novel',
       'jackson', 'films', 'love', 'post', 'ben', 'will', 'have',
       'something', 'time', 'tone', 'game', 'for', 'children', 'season',
       'more', 'kelly', 'episode', 'an', 'harry', 'mary', 'when',
       'batman', 'series', 'first', 'my', 'master', 'james', 'stop',
       'joe', 'jokes', 'heart', 'are', 'charlie', 'mad', 'full',
       'greatest', 'jane', 'show', 'steve', 'ever', 'but', 'with', 'they',
       'jason', 'bond', 'zombie', 'as', 'film', 'not', 'guy', 'movie',
       'sam', 'this', 'like', 'match', 'was', 'you', 'tony', 'she', 'her',
       'his', 'it', 'he', 'in', 'that', 'to', 'of', 'is', 'and', 'the'],
      dtype='<U14')

In [ ]:
max_value = X_train_tfidf_vect.max(axis=0).toarray().ravel()
sorted_value = max_value.argsort()
feature_names = np.array(tfidf_vect.get_feature_names())
feature_names[sorted_value[-100:]]

array(['king', 'gay', 'professional', 'hitchcock', 'acts', 'avoid',
       'german', 'band', 'no', 'bill', 'worse', 'season', 'terrible',
       'the', 'dancing', 'water', 'plenty', 'christian', 'car', 'hotel',
       'release', 'high', 'she', 'scott', 'cartoon', 'lame', 'guy',
       'jane', 'wall', 'indian', 'christopher', 'count', 'age', 'tone',
       'harry', 'song', 'college', 'opera', 'mark', 'disney', 'noir',
       'japanese', 'computer', 'hospital', 'version', 'heart', 'david',
       'jason', 'grace', 'henry', 'joe', 'kelly', 'tom', 'hot', 'dull',
       'think', 'view', 'cop', 'bond', 'tony', 'games', 'documentary',
       'pretty', 'sex', 'giant', 'wow', 'smith', 'wars', 'three',
       'zombie', 'cat', 'vs', 'ben', 'love', 'match', 'bed', 'stewart',
       'awful', 'check', 'ray', 'zombies', 'beautifully', 'charlie',
       'stupid', 'sam', 'mary', 'talk', 'game', 'and', 'batman',
       'martial', 'bruce', 'episode', 'master', 'buddy', 'bad', 'jokes',
       'horrible', 

#Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report #전체 레이블에 대해서
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score #각 레이블에 대해서

In [ ]:
def evaluation_report(y_test, pred, is_return=True):
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
        acc = accuracy_score(y_test, pred) 
        pre=precision_score(y_test, pred)
        recall = recall_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        auc = roc_auc_score(y_test, pred)
        for name, value in zip(['accuracy_score', 'precision_score', 'recall_score', 'f1_score', 'roc_auc_score'], [acc, pre, recall, f1, auc]):
            print('{name} = {value:.2f}'.format(name=name, value=value), end='\t')

# Example Model: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 200, n_jobs=-1)
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

In [ ]:
evaluation_report(y_test, pred)

accuracy_score = 0.88	precision_score = 0.88	recall_score = 0.89	f1_score = 0.88	roc_auc_score = 0.88	

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4993
           1       0.88      0.89      0.88      5007

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
lr_clf = LogisticRegression(max_iter = 200, n_jobs=-1)
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

In [ ]:
evaluation_report(y_test, pred)

accuracy_score = 0.89	precision_score = 0.88	recall_score = 0.89	f1_score = 0.89	roc_auc_score = 0.89	

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88      4993
           1       0.88      0.89      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



#Model Selection

In [ ]:
def model_selection(X_train, y_train, kfold=3):
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import LinearSVC
    from xgboost import XGBClassifier
    from sklearn.model_selection import cross_val_score
    from tqdm.notebook import tqdm

    models = [
        LogisticRegression(),
        XGBClassifier(random_state=random_seed),
        LinearSVC(),
    ]
    
    progress_bar = tqdm(total = len(models) * kfold)
    
    entries = []
    for model in models:
        model_name = model.__class__.__name__
        accuracies = cross_val_score(model, X_train, y_train, 
                        scoring='accuracy', cv=kfold, n_jobs=-1)
        #하나의 모델 당 총 3개의 accuracy값을 가지고 있는 리스트가 생성(kfold=3)
        for fold_idx, accuracy in enumerate(accuracies):
            progress_bar.update()
            entries.append((model_name, fold_idx, accuracy))
    
    progress_bar.close()
    
    cv_result = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'], index=range(kfold * len(models)))
    return cv_result

In [ ]:
model_selection_result = model_selection(X_train_tfidf_vect, y_train)

In [ ]:
model_selection_result

,model_name,fold_idx,accuracy
0,LogisticRegression,0,0.878656
1,LogisticRegression,1,0.872197
2,LogisticRegression,2,0.871447
3,XGBClassifier,0,0.813334
4,XGBClassifier,1,0.804995
5,XGBClassifier,2,0.810995
6,LinearSVC,0,0.876931
7,LinearSVC,1,0.871447
8,LinearSVC,2,0.871222


In [ ]:
pd.pivot_table(model_selection_result, values='accuracy', index=['model_name'], aggfunc=np.mean, fill_value=0)

,accuracy
model_name,
LinearSVC,0.873200
LogisticRegression,0.874100
XGBClassifier,0.809775


#Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=200)
params = {'penalty': ['l2', 'l1'],
       'C': [0.001, 0.01, 0.1, 1, 5, 10]}

grid_lr_clf = GridSearchCV(lr_clf, 
                           param_grid=params, verbose=1, scoring='accuracy', n_jobs=-1, cv = 3)

# eval_metric: 평가 방식. auc = accuracy
# eval_set: 평가 데이터셋
grid_lr_clf.fit(X_train_tfidf_vect, y_train) 

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   12.1s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=200, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 5, 10],
                         'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [ ]:
#최적 파라미터
grid_lr_clf.best_estimator_

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#최적 파라미터를 사용한 테스트셋 성능 확인
pred = grid_lr_clf.predict(X_test_cnt_vect)
evaluation_report(y_test, pred)

accuracy_score = 0.84	precision_score = 0.77	recall_score = 0.95	f1_score = 0.85	roc_auc_score = 0.84	

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.72      0.81      4993
           1       0.77      0.95      0.85      5007

    accuracy                           0.84     10000
   macro avg       0.85      0.84      0.83     10000
weighted avg       0.85      0.84      0.83     10000

